#  RAG 성능 평가 

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---

## RAG 시스템 성능 평가

- **RAG 기술의 핵심**: 외부 지식 검색과 LLM 결합으로 응답 품질 향상

- **평가 기준**: LLM-as-judge 방식으로 사실성, 관련성, 충실도, 유용성 평가

- **체계적인 A/B 테스트**: 각 컴포넌트별 성능 비교 및 영향도 분석으로 최적 구성 도출

- **평가 방법론**: 오프라인(참조답변 기반), 온라인(실시간), 페어와이즈(비교) 평가 구분

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_evaluation.png" alt="rag" align="center" border="0"  width="1000" height=auto>
</center>


---

### **평가 대상(Evaluation Target)**

- **검색(Retrieval)** 단계: 
    1. 관련 문서와 쿼리 간의 연관성(Relevance)을 통해 검색된 문서가 쿼리의 정보 요구를 얼마나 잘 충족하는지 평가
    1. 관련 문서와 후보 문서 간의 정확성(Accuracy)을 통해 시스템이 적절한 문서를 식별하는 능력을 측정

- **생성(Generation)** 단계:
    1. 응답과 쿼리의 연관성(Relavance)
    1. 응답과 관련 문서 간의 충실도(Faithfulness)
    1. 응답과 샘플 응답 간의 정확성(Correctness)

- 추가 고려사항:
    - **핵심 성능 지표**: Latency(응답 속도), Diversity(검색 다양성), Noise Robustness(잡음 내구성)
    - **안전성 평가**: Negative Rejection(불충분 정보 거부), Counterfactual Robustness(오정보 식별)
    - **사용자 경험**: Readability(가독성), Toxicity(유해성), Perplexity(복잡성) 등 추가 고려


<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_evaluation_target.png"  alt="rag" align="center" border="0"  width="800" height=auto>
</center>



[출처] https://arxiv.org/abs/2405.07437

---

### **평가 데이터셋 구축(Evaluation Dataset)**

- **데이터셋 구성 방식**: LLM 기반 새로운 데이터셋 생성 (Synthetic Data)

- **맞춤형 데이터셋** 구축으로 RAG 시스템의 실용성 평가 강화


- [실습] : **Ragas** (https://docs.ragas.io/en/stable/) 활용

    - **RAG 시스템 평가**를 위한 오픈소스 프레임워크
    - **주요 지표**: Faithfulness, Answer Relevancy, Context Relevancy 등 평가
    - **실용성**: 자동화된 평가 파이프라인 구축 가능

`(1) LangChain 문서 준비`

In [4]:
from langchain_community.document_loaders import TextLoader

# 데이터 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

korean_txt_files = glob(os.path.join('data', '*_KR.md')) 
korean_data = load_text_files(korean_txt_files)

print('Korean data:')
pprint(korean_data)

Korean data:
[Document(metadata={'source': 'data\\리비안_KR.md'}, page_content='Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.\n\n**주요 정보:**\n\n- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- **설립:** 2009년 6월, 플로리다 주 록ledge\n- **설립자:** R. J. 스캐린지\n- **본사:** 미국 캘리포니아 주 어바인\n- **서비스 지역:** 북미\n- **주요 인물:** R. J. 스캐린지 (CEO)\n- **제품:** 전기 자동차, 배터리\n- **생산량 (2023):** 57,232대\n- **서비스:** 전기 자동차 충전, 자동차 보험\n- **수익 (2023):** 44억 3천만 미국 달러\n- **순이익 (2023):** -54억 미국 달러\n- **총 자산 (2023):** 168억 미국 달러\n- **총 자본 (2023):** 91억 4천만 미국 달러\n- **직원 수 (2023년 12월):** 16,790명\n- **웹사이트:** rivian.com\n\n**개요**\n\nRivian은 "스케이트보드" 플랫폼(R1T 및 R1S 모델)을 기반으로 한 전기 스포츠 유틸리티 차량(SUV), 픽업 트럭 및 전기 배달 밴(Rivian EDV)을 생산합니다. R1T 배송은 2021년 말에 시작되었습니다. 회사는 2022년에 미국에서 충전 네트워크를 시작하여 2024년에 다른 차량에도 개방했습니다. 생산 공장은 일리노이 주 노멀에 있으며, 다른 시설은 미국, 캐나다, 영국 및 세르비아의 여러 주에 있습니다.\n\n**역사**\n\n**초창기 (2009–15):**\n\n- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립.\n- 2011년 Rivian Automotive로 사명 변경.\n- 처음에는 스포츠카 프로토타입(R1)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",    # TikToken 인코더 이름
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
    chunk_size=300,
    chunk_overlap=0,
    is_separator_regex=True,      # 구분자가 정규식인지 여부
    keep_separator=True,          # 구분자 유지 여부
)

korean_docs = text_splitter.split_documents(korean_data)

print("한국어 문서 수:", len(korean_docs))
print("-"*100)
print(korean_docs[0].metadata)
pprint(korean_docs[0].page_content)
print("-"*100)
print(korean_docs[1].metadata)
pprint(korean_docs[1].page_content)

한국어 문서 수: 39
----------------------------------------------------------------------------------------------------
{'source': 'data\\리비안_KR.md'}
('Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 '
 '회사입니다.\n'
 '\n'
 '**주요 정보:**')
----------------------------------------------------------------------------------------------------
{'source': 'data\\리비안_KR.md'}
('- **회사 유형:** 상장\n'
 '- **거래소:** NASDAQ: RIVN\n'
 '- **설립:** 2009년 6월, 플로리다 주 록ledge\n'
 '- **설립자:** R. J. 스캐린지\n'
 '- **본사:** 미국 캘리포니아 주 어바인\n'
 '- **서비스 지역:** 북미\n'
 '- **주요 인물:** R. J. 스캐린지 (CEO)\n'
 '- **제품:** 전기 자동차, 배터리\n'
 '- **생산량 (2023):** 57,232대\n'
 '- **서비스:** 전기 자동차 충전, 자동차 보험\n'
 '- **수익 (2023):** 44억 3천만 미국 달러\n'
 '- **순이익 (2023):** -54억 미국 달러\n'
 '- **총 자산 (2023):** 168억 미국 달러')


In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성하기
vector_store = Chroma.from_documents(
    documents=korean_docs,
    embedding=embedding_model,    
    collection_name="db_korean_cosine", 
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

저장된 Document 개수: 39


`(2) LLM 설정`

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini", temperature=0.2))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

`(3) Test Data 생성`

- uv add rapidfuzz 설치 필요

In [9]:
from ragas.testset.persona import Persona

# 페르소나 정의 (다양한 관점에서 질문 생성)
personas = [
    Persona(
        name="graduate_researcher",  # 박사과정 연구원: 심도 있는 분석적 질문
        role_description="미국 전기차 시장을 연구하는 한국인 박사과정 연구원으로, 전기차 정책과 시장 동향에 대해 깊이 있는 분석을 하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="masters_student",    # 석사과정 학생: 개념 이해를 위한 질문
        role_description="전기차 산업을 공부하는 한국인 석사과정 학생으로, 미국 전기차 시장의 기초적인 개념과 트렌드를 이해하려 노력하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="industry_analyst",   # 산업 분석가: 실무 중심적 질문
        role_description="한국 자동차 회사에서 미국 전기차 시장을 분석하는 주니어 연구원으로, 실무적인 시장 데이터와 경쟁사 동향에 관심이 많습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="undergraduate_student",  # 학부생: 기초적인 학습 질문
        role_description="자동차 공학을 전공하는 한국인 학부생으로, 미국 전기차 기술과 시장에 대해 기본적인 지식을 습득하고자 합니다. 한국어만을 사용합니다.",
    )
]

In [10]:
from ragas.testset import TestsetGenerator

# TestsetGenerator 생성
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)

# 합성 데이터 생성
dataset = generator.generate_with_langchain_docs(korean_docs, testset_size=5)

Applying CustomNodeFilter:   0%|          | 0/39 [00:00<?, ?it/s]         Node bf12ac9c-cd91-4c72-9d16-e6f6ee41441f does not have a summary. Skipping filtering.
Node 611b190b-233e-4dba-a75f-ccba5077895c does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  28%|██▊       | 11/39 [00:00<00:01, 19.94it/s]Node ff357ea4-caf7-4f6e-b60c-28d1b7328cdd does not have a summary. Skipping filtering.
Node 7f613de4-0e42-42fe-8f3a-e1a8e34b06ea does not have a summary. Skipping filtering.
Generating Samples: 100%|██████████| 6/6 [00:03<00:00,  1.61it/s]


In [11]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Can you explane in detial about Rivian Automot...,"[Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동...","Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차...",single_hop_specifc_query_synthesizer
1,2023년 리비안(Rivian)의 재무 성과 중에서 순이익이 -54억 미국 달러인 ...,[- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- *...,"2023년 리비안은 44억 3천만 미국 달러의 수익을 올렸지만, 순이익은 -54억 ...",single_hop_specifc_query_synthesizer
2,why tesla move legal headquarters to gigafacto...,"[<1-hop>\n\n2023년 3월, Tesla는 잠재적인 관세 영향으로 인해 지...",Tesla moved its legal headquarters to Gigafact...,multi_hop_abstract_query_synthesizer
3,when Model X delivery start and Cybertruck del...,"[<1-hop>\n\n- **Model S:** 리프트백 차체 스타일과 듀얼 모터,...","Model X delivery started in September 2015, an...",multi_hop_abstract_query_synthesizer
4,2023년 기준으로 미국 전기차 시장에서 NASDAQ에 상장된 Rivian의 생산량...,[<1-hop>\n\n- **회사 유형:** 상장\n- **거래소:** NASDAQ...,2023년 기준으로 Rivian은 미국 캘리포니아 주 어바인에 본사를 둔 상장 기업...,multi_hop_specific_query_synthesizer
5,"Rivian 2021년 9월에 R1T 배송 시작했는데, 2021년 전체 상황하고 어...",[<1-hop>\n\n- R1T 배송은 2021년 9월에 시작되어 Rivian은 완...,Rivian은 2021년 9월에 R1T 배송을 시작하여 완전 전기 픽업을 소비자 시...,multi_hop_specific_query_synthesizer


In [12]:
# 데이터 저장
dataset.to_pandas().to_csv('./data/ragas_testset.csv', index=False)

---

### **평가 지표(Evaluation Metric)**

#### 1) **검색(Retrieval) 평가**  

- **Non-Rank Based Metrics**: Accuracy, Precision, Recall@k 등을 통해 관련성의 이진적 평가를 수행

- **Rank-Based Metrics**: MRR(Mean Reciprocal Rank), MAP(Mean Average Precision)를 통해 검색 결과의 순위를 고려한 평가를 수행

- **RAG 특화 지표**: 기존 검색 평가 방식의 한계를 보완하는 LLM-as-judge 방식 도입

- **포괄적 평가**: 정확도, 관련성, 다양성, 강건성을 통합적으로 측정

#### 2) **생성(Generation) 평가**

- **전통적 평가**: ROUGE(요약), BLEU(번역), BertScore(의미 유사도) 지표 활용

- **LLM 기반 평가**: 응집성, 관련성, 유창성을 종합적으로 판단하는 새로운 접근법 도입 (전통적인 참조 비교가 어려운 상황에서 유용)

- **다차원 평가**: 품질, 일관성, 사실성, 가독성, 사용자 만족도를 포괄적 측정

- **상세 프롬프트**와 **사용자 선호도** 기준으로 생성 텍스트 품질 평가

`(1) RAG 체인 - 평가 대상`

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 벡터 저장소 검색기 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# RAG 체인 
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 템플릿 생성
template = """Answer the question based only on the following context:
{context}

Question: {query}
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain = prompt | llm | StrOutputParser()

def format_docs(relevant_docs):
    return "\n".join(doc.page_content for doc in relevant_docs)


query = "Tesla는 언제 누가 만들었나?"

relevant_docs = retriever.invoke(query)
qa_chain.invoke({"context": format_docs(relevant_docs), "query": query})

'Tesla는 2003년 7월 1일에 Martin Eberhard와 Marc Tarpenning에 의해 설립되었습니다.'

`(2) 평가 수행을 위한 데이터셋 전처리`

In [16]:
# 데이터 로드
import pandas as pd
testset = pd.read_excel('data/testset.xlsx')

# 데이터 확인
testset.head()

,user_input,reference_contexts,reference,synthesizer_name
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","['Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(...",single_hop_specifc_query_synthesizer
1,Forbes Global 2000에서 테슬라 순위 뭐야?,['Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (...,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.,single_hop_specifc_query_synthesizer
2,Tesla는 언제 누가 만들었나?,"['Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, M...","Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberha...",single_hop_specifc_query_synthesizer
3,Larry Page는 전기차 시장에서 어떤 역할을 했나요?,"[""Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전...",Larry Page는 전기차 시장에서 Tesla의 후속 자금 조달에 투자한 기업가 ...,single_hop_specifc_query_synthesizer
4,Toyota와 Tesla의 관계는 무엇인가요?,['Roadster 생산은 2008년에 시작되었습니다. 2009년 1월까지 Tesl...,"2010년 5월, Tesla는 캘리포니아 주 프리몬트의 NUMMI 공장을 Toyot...",single_hop_specifc_query_synthesizer


In [17]:
from ragas import EvaluationDataset

# 데이터셋 생성
dataset = []

# 각 행에 대해 RAG 체인을 호출하여 결과를 저장
for row in testset.itertuples():
    query = row.user_input   # 사용자 입력
    reference = row.reference  # 참조 답변
    relevant_docs = retriever.invoke(query)  # 검색된 문서
    response = qa_chain.invoke(      # RAG 체인 생성 답변 생성
        {
            "context": format_docs(relevant_docs),
            "query": query,
        }
    )  
    
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [d.page_content for d in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [18]:
# 데이터프레임 변환하여 확인 
evaluation_dataset.to_pandas().head()

,user_input,retrieved_contexts,response,reference
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","[Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로서 전기 자동차(...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(..."
1,Forbes Global 2000에서 테슬라 순위 뭐야?,[Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (2...,Forbes Global 2000에서 Tesla의 순위는 69위입니다.,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.
2,Tesla는 언제 누가 만들었나?,"[Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Mu...",Tesla는 2003년 7월 1일에 Martin Eberhard와 Marc Tarp...,"Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberha..."
3,Larry Page는 전기차 시장에서 어떤 역할을 했나요?,[Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략...,Larry Page는 Tesla의 후속 자금 조달에 투자자로 참여하여 전기차 시장에...,Larry Page는 전기차 시장에서 Tesla의 후속 자금 조달에 투자한 기업가 ...
4,Toyota와 Tesla의 관계는 무엇인가요?,[## 파트너\n\nTesla는 Panasonic과 파트너십을 맺고 있으며 리튬 공...,Toyota와 Tesla는 과거 파트너 관계에 있었습니다.,"2010년 5월, Tesla는 캘리포니아 주 프리몬트의 NUMMI 공장을 Toyot..."


In [19]:
# 데이터 저장
evaluation_dataset.to_pandas().to_csv('data/evaluation_dataset.csv', index=False)

`(3) 평가 수행`

- **LLMContextRecall**
    - 검색된 컨텍스트가 정답을 생성하는 데 필요한 **모든 정보를 포함**하고 있는지 평가

- **Faithfulness** 
    - 생성된 응답이 검색된 컨텍스트와 얼마나 **사실적으로 일관**되는지를 측정

- **FactualCorrectness**
    - 생성된 응답과 reference의 **사실적 정확**성을 비교하고 평가

In [20]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from langchain_openai import ChatOpenAI

# LLM 래퍼 생성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)

# 평가
result = evaluate(
    dataset=evaluation_dataset,   # 평가 데이터셋
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],   # 평가 메트릭
    llm=evaluator_llm,   # LLM 래퍼
)

result

Evaluating: 100%|██████████| 147/147 [02:01<00:00,  1.21it/s]


{'context_recall': 0.6803, 'faithfulness': 0.8933, 'factual_correctness(mode=f1)': 0.5988}

In [21]:
# 결과를 데이터프레임으로 변환 
result.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","[Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로서 전기 자동차(...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(...",1.000000,1.000000,1.00
1,Forbes Global 2000에서 테슬라 순위 뭐야?,[Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (2...,Forbes Global 2000에서 Tesla의 순위는 69위입니다.,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.,1.000000,1.000000,1.00
2,Tesla는 언제 누가 만들었나?,"[Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Mu...",Tesla는 2003년 7월 1일에 Martin Eberhard와 Marc Tarp...,"Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberha...",1.000000,1.000000,0.67
3,Larry Page는 전기차 시장에서 어떤 역할을 했나요?,[Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략...,Larry Page는 Tesla의 후속 자금 조달에 투자자로 참여하여 전기차 시장에...,Larry Page는 전기차 시장에서 Tesla의 후속 자금 조달에 투자한 기업가 ...,1.000000,0.500000,0.80
4,Toyota와 Tesla의 관계는 무엇인가요?,[## 파트너\n\nTesla는 Panasonic과 파트너십을 맺고 있으며 리튬 공...,Toyota와 Tesla는 과거 파트너 관계에 있었습니다.,"2010년 5월, Tesla는 캘리포니아 주 프리몬트의 NUMMI 공장을 Toyot...",0.000000,1.000000,0.00
5,Tesla의 Model 3에 대한 정보를 어디서 찾을 수 있나요?,[Tesla는 2016년 11월 SolarCity를 26억 달러에 인수하여 Tesl...,"Tesla의 Model 3에 대한 정보는 제공된 문서 내 ""Model 3 세단은 2...",제공된 문맥에는 Tesla의 Model 3에 대한 정보가 포함되어 있지 않습니다.,0.000000,1.000000,0.00
6,Tesla Energy 뭐야?,[Tesla Energy는 태양 에너지 생성 시스템과 배터리 에너지 저장 제품을 개...,Tesla Energy는 태양 에너지 생성 시스템과 배터리 에너지 저장 제품을 개발...,Tesla는 2016년 11월 SolarCity를 26억 달러에 인수하여 Tesla...,0.000000,1.000000,0.00
7,2020년 3월에 Tesla의 주요 사건은 무엇입니까?,[Tesla는 2016년 11월 SolarCity를 26억 달러에 인수하여 Tesl...,제공된 문맥에는 2020년 3월에 Tesla의 주요 사건에 대한 정보가 포함되어 있...,2020년 3월에 Tesla는 Model Y 중형 크로스오버 SUV의 배송을 시작했...,0.000000,1.000000,0.00
8,Megafactory가 Tesla의 사업 확장에 어떻게 기여했는지 설명해줄 수 있나요?,"[2023년 3월, Tesla는 잠재적인 관세 영향으로 인해 지연된 Gigafact...","Megafactory는 2022년에 설립된 Tesla의 새로운 시설로, 캘리포니아에...",Tesla는 2022년에 Megafactory를 통해 캘리포니아에서의 입지를 확장했...,1.000000,0.833333,0.33
9,Tesla가 Bitcoin에 투자하고 나중에 매각한 이유는 뭐였나요?,"[2021년 초, Tesla는 Bitcoin에 15억 달러를 투자하고 환경 문제로 ...",Tesla가 Bitcoin에 투자한 이유는 2021년 초에 15억 달러를 투자하고 ...,"Tesla는 2021년 초에 Bitcoin에 15억 달러를 투자했으나, 환경 문제로...",1.000000,0.600000,0.89


In [22]:
# 데이터프레임 저장
result.to_pandas().to_csv('data/evaluation_result.csv', index=False)